# Neural Network Embeddings - Solution

In [ ]:
import tensorflow as tf
import pandas as pd

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

### Tweet Emotion data

This is a dataset of 40,000 tweets labeled with one of 13 emotions:
['empty', 'sadness', 'enthusiasm', 'neutral', 'worry', 'surprise',
      'love', 'fun', 'hate', 'happiness', 'boredom', 'relief', 'anger']
      
Tokenize the data and train it, with a neural network, to  predict the category of a given tweet. 

In [ ]:
text_data = pd.read_csv("text_emotion.csv")

# transform string labels to integers
text_data["sentiment"] = text_data["sentiment"].map({'empty':0, 'sadness':1, 'enthusiasm':2, 'neutral':3, 'worry':4, 'surprise':5,'love':6, 'fun':7, 'hate':8, 'happiness':9, 'boredom':10, 'relief':11, 'anger':12})

# Text data
train_text = text_data["content"].values[:20000]
test_text = text_data["content"].values[20000:]

# Labels
train_text_labels = text_data["sentiment"].values[:20000]
test_text_labels = text_data["sentiment"].values[20000:]

In [ ]:
# preview some of the tweets
print(train_text[:5])

### 1.)  Tokenize the dataset

In [ ]:
tokenizer = Tokenizer(num_words=10000, oov_token="<oov>")
tokenizer.fit_on_texts(train_text)

word_index = tokenizer.word_index

training_sequences = tokenizer.texts_to_sequences(train_text)
training_padded = pad_sequences(training_sequences, maxlen=32, 
                                padding="post", truncating="post")

testing_sequences = tokenizer.texts_to_sequences(test_text)
testing_padded = pad_sequences(testing_sequences, maxlen=32, 
                                padding="post", truncating="post")

### 2.) Create a Neural Network with embeddings

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(10000, 16, input_length=32),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(24, activation = 'relu'),
    tf.keras.layers.Dense(13, activation = 'softmax')
])

model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

### 3.) Train the model

In [ ]:

history = model.fit(training_padded, train_text_labels, epochs=30, 
                    validation_split=.1, verbose=2)


### Plot the loss

In [ ]:
import matplotlib.pyplot as plt

def plot_graphs(history, string):
    plt.plot(history.history[string])
    plt.plot(history.history['val_'+string])
    plt.title('Training and validation')
    plt.xlabel('Epochs')
    plt.ylabel(string)
    plt.legend([string, 'val_'+string])
    plt.show()

plot_graphs(history, "accuracy")
plot_graphs(history, "loss")